In [1]:
import sys
sys.path.append("../src")
sys.path.append("/home/niyiyu/notebooks/DASStore")

import os
import time
import pyasdf
from datetime import datetime, timedelta
import h5py
import numpy as np
import DAS_module
import pandas as pd
import matplotlib.pyplot as plt

from dasstore.zarr import Client

In [3]:
# client information
bucket = "seadas-december-2022"
endpoint = "s3.us-west-2.amazonaws.com"
secure = True
role_assigned = True

# prepare client and time range
client = Client(bucket, endpoint, secure = secure, credential_path="~/.aws/credentials")

KeyError: 'No credential found for endpoint s3.us-west-2.amazonaws.com'

In [3]:
# useful parameters for preprocessing the data
input_fmt = 'zarr'                                                      # input file format between 'sac' and 'mseed' 
sps       = 100                                                         # current sampling rate
samp_freq = 50                                                          # targeted sampling rate
freqmin   = 1                                                          # pre filtering frequency bandwidth
freqmax   = 20                                                          # note this cannot exceed Nquist freq
flag      = True                                                        # print intermediate variables and computing time
gaug_len  = 2                                                           # gauge length of the array for inter-station distance (assuming linear array)

# useful parameters for cross correlating the data
freq_norm   = 'phase_only'                                                     # 'no' for no whitening, or 'rma' for running-mean average, 'phase_only' for sign-bit normalization in freq domain.
time_norm   = 'one_bit'                                                 # 'no' for no normalization, or 'rma', 'one_bit' for normalization in time domain
cc_method   = 'xcorr'                                                   # 'xcorr' for pure cross correlation, 'deconv' for deconvolution; FOR "COHERENCY" PLEASE set freq_norm to "rma", time_norm to "no" and cc_method to "xcorr"
smooth_N    = 100                                                       # moving window length for time domain normalization if selected (points)
smoothspect_N  = 100                                                    # moving window length to smooth spectrum amplitude (points)
maxlag      = 10                                                        # lags of cross-correlation to save (sec)

# criteria for data selection
max_over_std = 10                                                       # threahold to remove window of bad signals: set it to 10*9 if prefer not to remove them

# maximum memory allowed per core in GB
MAX_MEM   = 4.0        

In [4]:
cc_len = 60
step   = 60

N_NODE = 10
NODE_ID = 2

In [6]:
t0 = datetime.fromisoformat(client.meta['acquisition.acquisition_start_time']).date()
t1 = datetime.fromisoformat(client.meta['acquisition.acquisition_end_time']).date()
n_days = (t1 - t0 + timedelta(days=1)).days
date_list = [t0 + timedelta(days = i) for i in range(n_days)]
split = np.array_split(date_list, N_NODE)[NODE_ID]

In [7]:
# load one sample file to get some basic parameters
cha_list = np.array(range(500, 1100))  

In [8]:
CCFDIR = "/home/niyiyu/Research/SeaDASCorr/DASStore_CCF"
prepro_para = {'CCFDIR':CCFDIR,
               'input_fmt':input_fmt,
               'freqmin':freqmin,
               'freqmax':freqmax,
               'sps':sps,
               'npts_chunk':cc_len*sps,
               'nsta':len(cha_list),
               'cha_list':cha_list,
               'samp_freq':samp_freq,
            #    'allfiles_path':rootpath,
               'freq_norm':freq_norm,
               'time_norm':time_norm,
               'cc_method':cc_method,
               'smooth_N':smooth_N,
               'smoothspect_N':smoothspect_N,
               'maxlag':maxlag,
               'max_over_std':max_over_std,
               'MAX_MEM':MAX_MEM}
metadata = os.path.join(CCFDIR,'prepro_fft_info.txt')
# output parameter info
fout = open(metadata,'w')
fout.write(str(prepro_para));fout.close()

In [25]:
for i in split:
    # one node
    node_t0 = datetime(year = i.year, month = i.month, day = i.day, 
                       hour = 0, minute = 0, second = 0, microsecond=0)
    node_t1 = datetime(year = i.year, month = i.month, day = i.day + 1, 
                    hour = 0, minute = 0, second = 0, microsecond=0)

    print(node_t0, node_t1)

2022-12-08 00:00:00 2022-12-09 00:00:00
2022-12-09 00:00:00 2022-12-10 00:00:00
2022-12-10 00:00:00 2022-12-11 00:00:00


In [10]:
ick = 0
tdata = client.get_data(cha_list, 
            starttime = node_t0,
            endtime = node_t0 + timedelta(minutes = 1)).T
# temp  = allfiles[ick].split('/')[-1].split('_')
# tvec  = str(temp[3]+temp[4].split('.')[0])
# print(tvec)

# perform pre-processing
trace_stdS,dataS = DAS_module.preprocess_raw_make_stat(tdata,prepro_para)

# do normalization if needed
white_spect = DAS_module.noise_processing(dataS,prepro_para)
Nfft = white_spect.shape[1];Nfft2 = Nfft//2

# load fft data in memory for cross-correlations
data = white_spect[:,:Nfft2]
del dataS,white_spect

In [14]:
# remove channel of potential local traffic noise
ind = np.where((trace_stdS<prepro_para['max_over_std']) &
                (trace_stdS>0) &
                (np.isnan(trace_stdS)==0))[0]
if not len(ind):
    raise ValueError('the max_over_std criteria is too high which results in no data')
sta = cha_list[ind]
white_spect = data[ind]

In [23]:
ftmp = open(CCFDIR + "/tmp.log",'w')
for iiS in range(len(sta)-1):
# for iiS in range(3):
    sfft1 = DAS_module.smooth_source_spect(white_spect[iiS],prepro_para)
    corr,tindx = DAS_module.correlate(sfft1,white_spect[iiS+1:],prepro_para,Nfft)
    # update the receiver list
    tsta = sta[iiS+1:]
    receiver_lst = sta[tindx]

    # save cross-correlation into ASDF file
    for iiR in range(len(receiver_lst)):
        with pyasdf.ASDFDataSet(CCFDIR + "/test.h5", mpi=False) as ccf_ds:
            # use the channel number as a way to figure out distance
            Rindx = np.where(sta==receiver_lst[iiR])[0]
            Sindx = iiS
            param = {'dist':int(Rindx-Sindx)*gaug_len,
                    'sps':samp_freq,
                    'dt': 1/samp_freq,
                    'maxlag':maxlag,
                    'freqmin':freqmin,
                    'freqmax':freqmax}
        
            # source-receiver pair
            data_type = str(sta[iiS])
            path = str(sta[iiS])+'_'+str(receiver_lst[iiR])
            ccf_ds.add_auxiliary_data(data=corr[iiR], 
                                        data_type=data_type, 
                                        path=path, 
                                        parameters=param)
            ftmp.write(str(sta[iiS])+'_'+str(receiver_lst[iiR])+'\n')